# 필요 패키지 로드

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
data1 = pd.read_csv('label_1108.csv', encoding = 'utf-8')

In [3]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8074 entries, 0 to 8073
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8074 non-null   int64  
 1   icls_cnts   8074 non-null   object 
 2   label       8074 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 189.4+ KB


In [4]:
data1 = data1[['label','icls_cnts']]
data1['label'].value_counts()

 0.0    5216
 1.0    1936
-1.0     922
Name: label, dtype: int64

In [5]:
data1['label'].replace(999,0,inplace = True)
data1['label'].replace(-1,1, inplace =True)
data1['label'].value_counts()

0.0    5216
1.0    2858
Name: label, dtype: int64

In [6]:
data1.drop_duplicates(subset = ['icls_cnts'], inplace = True)
data1['label'].value_counts()

0.0    5216
1.0    2858
Name: label, dtype: int64

In [7]:
df1 = data1

In [8]:
print('결측값 여부:',df1.isnull().values.any())

결측값 여부: False


In [9]:
import re 
#텍스트 정제 함수 : 한글 이외의 문자 제거
def text_cleaning(text):
    #한글 정규표현식으로 한글만 추출
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+')
    result = hangul.sub('',text)
    return result

In [10]:
df1['ko_text']=df1['icls_cnts'].apply(lambda x: text_cleaning(x))
df1.head

<bound method NDFrame.head of       label                                          icls_cnts  \
0       1.0  ..<b>경찰</b>과 추격전까지 벌인 40대 남성이 붙잡혔습니다. .. 1시간이 ...   
1       1.0  수원남부署, 모범<b>경찰</b> 최보라·김세홍 경장에 표창·장려장 수원남부<b>경...   
2       1.0   조두순에 대한 1대1 전자감독 및 <b>경찰</b>을 통한 24시간 밀착 감독 등...   
3       1.0  순천<b>경찰</b>서, 전화금융사기 예방 홍보시스템 구축 순천<b>경찰</b>서는...   
4       1.0  경남<b>경찰</b>청 직원들, 사랑의 헌혈 릴레이 운동 경남지방<b>경찰</b>청...   
...     ...                                                ...   
8069    0.0  ‘LH 땅투기 <b>수사</b>’ 검·경 실무진 첫 회동…”검찰 들러리 세우는 것”...   
8070    1.0  "인천 흉기난동" 부실대응 <b>경찰</b>관 송치 인천<b>경찰</b>청 광역<b...   
8071    0.0   일부 산부인과의 진료기록 공개 거부 등 반발을 우려해 법원으로부터 압수수색 영장까...   
8072    0.0  ‘마우스’ 경수진, 털털하고 냉철한 PD 변신 배우 경수진이 tvN 드라마 ..‘마...   
8073    0.0  “번호 지워서” 16살 연하 남친 살해 여성, 무기징역→징역 22년 감형…왜? 본인...   

                                                ko_text  
0     경찰과추격전까지벌인대남성이붙잡혔습니다시간이넘도록음주운전차량을뒤쫓던경찰은결국실탄을쏴시...  
1     수원남부모범경찰최보라김세홍경장에표창장려장수원남부경찰서서장오문교는중요범인검거유공등타의...  
2  

In [11]:
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word,tag) for word, tag in pos]
    return pos

#함수 테스트 
result = get_pos(df1['ko_text'][0])
print(result)

['경찰/Noun', '과/Josa', '추/Noun', '격전/Noun', '까지/Josa', '벌인대/Verb', '남성/Noun', '이/Josa', '붙잡혔습니다/Verb', '시간/Noun', '이/Josa', '넘도록/Verb', '음주운전/Noun', '차량/Noun', '을/Josa', '뒤/Noun', '쫓던/Verb', '경찰/Noun', '은/Josa', '결국/Adverb', '실탄/Noun', '을/Josa', '쏴/Verb', '시간/Noun', '이/Josa', '넘도록/Verb', '음주운전/Noun', '차량/Noun', '을/Josa', '뒤/Noun', '쫓던/Verb', '경찰/Noun', '은/Josa', '결국/Adverb', '실탄/Noun', '을/Josa', '쏴/Verb', '차/Noun', '를/Josa', '세웠는데/Verb', '잡고/Noun', '보니이/Verb', '남성/Noun', '운전면허/Noun', '가/Josa', '없었습니다/Adjective', '경찰서/Noun', '경찰/Noun', '은/Josa', '도로/Noun', '교통법위반/Noun', '등/Noun', '의/Josa', '혐의/Noun', '로/Josa', '살/Noun', '씨/Suffix', '에/Josa', '대해/Noun', '구속영장/Noun', '을/Josa', '신청/Noun', '할/Verb', '계획/Noun', '입니다/Adjective']


In [12]:
index_vectorizer = CountVectorizer(tokenizer = lambda x : get_pos(x))
X = index_vectorizer.fit_transform(df1['ko_text'].tolist())

In [13]:
X.shape

(8074, 35370)

In [14]:
print(df1['ko_text'][0])
print(X[0])

경찰과추격전까지벌인대남성이붙잡혔습니다시간이넘도록음주운전차량을뒤쫓던경찰은결국실탄을쏴시간이넘도록음주운전차량을뒤쫓던경찰은결국실탄을쏴차를세웠는데잡고보니이남성운전면허가없었습니다경찰서경찰은도로교통법위반등의혐의로살씨에대해구속영장을신청할계획입니다
  (0, 1765)	4
  (0, 2367)	1
  (0, 30413)	1
  (0, 1530)	1
  (0, 4219)	1
  (0, 13707)	1
  (0, 5052)	2
  (0, 24037)	3
  (0, 15266)	1
  (0, 18629)	2
  (0, 5553)	2
  (0, 23870)	2
  (0, 29377)	2
  (0, 23820)	5
  (0, 8690)	2
  (0, 29269)	2
  (0, 23735)	3
  (0, 1581)	2
  (0, 19163)	2
  (0, 19445)	2
  (0, 29347)	1
  (0, 9976)	1
  (0, 17571)	1
  (0, 26343)	1
  (0, 14083)	1
  (0, 22962)	1
  (0, 4)	1
  (0, 21242)	1
  (0, 1779)	1
  (0, 7555)	1
  (0, 2708)	1
  (0, 9138)	1
  (0, 23896)	1
  (0, 34439)	1
  (0, 9836)	1
  (0, 16324)	1
  (0, 19549)	1
  (0, 21292)	1
  (0, 7324)	1
  (0, 2816)	1
  (0, 19070)	1
  (0, 33307)	1
  (0, 1856)	1
  (0, 25685)	1


In [15]:
tfidf_vectorizer = TfidfTransformer()
X = tfidf_vectorizer.fit_transform(X)

In [16]:
print(X.shape)
print(X[0])

(8074, 35370)
  (0, 34439)	0.056720501413239165
  (0, 33307)	0.06759964537134064
  (0, 30413)	0.12642961523183818
  (0, 29377)	0.1666966856318872
  (0, 29347)	0.09697326658404562
  (0, 29269)	0.3840522635730061
  (0, 26343)	0.14073649703171057
  (0, 25685)	0.10008632152629426
  (0, 24037)	0.09388562836587759
  (0, 23896)	0.03160157510643609
  (0, 23870)	0.2013904580574777
  (0, 23820)	0.11622090759890523
  (0, 23735)	0.09267680887858244
  (0, 22962)	0.14349264739115963
  (0, 21292)	0.025727903548247014
  (0, 21242)	0.17771924998663066
  (0, 19549)	0.06784983591853408
  (0, 19445)	0.2911168335142245
  (0, 19163)	0.2503565894003772
  (0, 19070)	0.09463410954684548
  (0, 18629)	0.1713195690154195
  (0, 17571)	0.19202613178650305
  (0, 16324)	0.10894099134671784
  (0, 15266)	0.14253245252391808
  (0, 14083)	0.19202613178650305
  (0, 13707)	0.19202613178650305
  (0, 9976)	0.027124867560636406
  (0, 9836)	0.04083843304212885
  (0, 9138)	0.041734928201384976
  (0, 8690)	0.1578924569187968
  (

In [17]:
from sklearn.model_selection import train_test_split

y = df1['label']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1)
print(X_train.shape)
print(X_test.shape)

(7266, 35370)
(808, 35370)


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_pred_probability = lr.predict_proba(X_test)[:,1]

print("정확도:%.2f"% accuracy_score(y_test, y_pred))
print("정밀도:%.3f"% precision_score(y_test,y_pred, average = 'weighted'))
print("재현율:%.3f"% recall_score(y_test, y_pred, average = 'weighted'))
print("F1:%.3f"% f1_score(y_test,y_pred, average = 'weighted'))

정확도:0.77
정밀도:0.765
재현율:0.771
F1:0.760


# 데이터 전처리

In [19]:
data = pd.read_csv('label_1108.csv', encoding = 'utf-8')

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8074 entries, 0 to 8073
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8074 non-null   int64  
 1   icls_cnts   8074 non-null   object 
 2   label       8074 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 189.4+ KB


In [21]:
data['label'].replace(999,0,inplace = True)

In [22]:
data = data[['icls_cnts','label']]
data

,icls_cnts,label
0,..<b>경찰</b>과 추격전까지 벌인 40대 남성이 붙잡혔습니다. .. 1시간이 ...,1.0
1,"수원남부署, 모범<b>경찰</b> 최보라·김세홍 경장에 표창·장려장 수원남부<b>경...",1.0
2,조두순에 대한 1대1 전자감독 및 <b>경찰</b>을 통한 24시간 밀착 감독 등...,1.0
3,"순천<b>경찰</b>서, 전화금융사기 예방 홍보시스템 구축 순천<b>경찰</b>서는...",1.0
4,"경남<b>경찰</b>청 직원들, 사랑의 헌혈 릴레이 운동 경남지방<b>경찰</b>청...",1.0
...,...,...
8069,‘LH 땅투기 <b>수사</b>’ 검·경 실무진 첫 회동…”검찰 들러리 세우는 것”...,0.0
8070,"""인천 흉기난동"" 부실대응 <b>경찰</b>관 송치 인천<b>경찰</b>청 광역<b...",-1.0
8071,일부 산부인과의 진료기록 공개 거부 등 반발을 우려해 법원으로부터 압수수색 영장까...,0.0
8072,"‘마우스’ 경수진, 털털하고 냉철한 PD 변신 배우 경수진이 tvN 드라마 ..‘마...",0.0


In [23]:
label_list = [-1,1]
data = data[data['label'].isin(label_list)]
data['label'].value_counts()

 1.0    1936
-1.0     922
Name: label, dtype: int64

In [24]:
data.drop_duplicates(subset = ['icls_cnts'], inplace = True)
data['label'].value_counts()

/tmp/ipykernel_132531/1251076988.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop_duplicates(subset = ['icls_cnts'], inplace = True)


 1.0    1936
-1.0     922
Name: label, dtype: int64

In [25]:
df = data

In [26]:
print('결측값 여부:',df.isnull().values.any())

결측값 여부: False


# 문장 정제 및 토큰화

In [27]:
import re 
#텍스트 정제 함수 : 한글 이외의 문자 제거
def text_cleaning(text):
    #한글 정규표현식으로 한글만 추출
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+')
    result = hangul.sub('',text)
    return result

In [28]:
#만든 함수 적용
df['ko_text']=df['icls_cnts'].apply(lambda x: text_cleaning(x))
df.head

/tmp/ipykernel_132531/2696196206.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ko_text']=df['icls_cnts'].apply(lambda x: text_cleaning(x))


<bound method NDFrame.head of                                               icls_cnts  label  \
0     ..<b>경찰</b>과 추격전까지 벌인 40대 남성이 붙잡혔습니다. .. 1시간이 ...    1.0   
1     수원남부署, 모범<b>경찰</b> 최보라·김세홍 경장에 표창·장려장 수원남부<b>경...    1.0   
2      조두순에 대한 1대1 전자감독 및 <b>경찰</b>을 통한 24시간 밀착 감독 등...    1.0   
3     순천<b>경찰</b>서, 전화금융사기 예방 홍보시스템 구축 순천<b>경찰</b>서는...    1.0   
4     경남<b>경찰</b>청 직원들, 사랑의 헌혈 릴레이 운동 경남지방<b>경찰</b>청...    1.0   
...                                                 ...    ...   
8054  ‘경찰 현장 대응력 강화 태스크포스(TF)’는 4차 회의에서 이 같은 내용을 결정했...    1.0   
8058  차에 불지르고 흉기 난동…30대 테이저건으로 제압 대구에서 흉기로 행인을 위협하고 ...    1.0   
8064  [사설] 범인 제압 훈련을 온라인으로 때웠다는 경찰 대통령의 질타와 경찰청장의 사과...   -1.0   
8065  울산<b>경찰</b>, 방역수칙 위반 유흥주점 업주와 손님 등 9명 입건 울산<b>...    1.0   
8070  "인천 흉기난동" 부실대응 <b>경찰</b>관 송치 인천<b>경찰</b>청 광역<b...   -1.0   

                                                ko_text  
0     경찰과추격전까지벌인대남성이붙잡혔습니다시간이넘도록음주운전차량을뒤쫓던경찰은결국실탄을쏴시...  
1     수원남부모범경찰최보라김세홍경장에표창장려장수원남부경찰서서장오문교는중요범인검거유공등타의...  
2  

In [29]:
#okt 라이브러리로 텍스트 데이터에서 형태소 추출
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word,tag) for word, tag in pos]
    return pos

#함수 테스트 
result = get_pos(df['ko_text'][0])
print(result)

['경찰/Noun', '과/Josa', '추/Noun', '격전/Noun', '까지/Josa', '벌인대/Verb', '남성/Noun', '이/Josa', '붙잡혔습니다/Verb', '시간/Noun', '이/Josa', '넘도록/Verb', '음주운전/Noun', '차량/Noun', '을/Josa', '뒤/Noun', '쫓던/Verb', '경찰/Noun', '은/Josa', '결국/Adverb', '실탄/Noun', '을/Josa', '쏴/Verb', '시간/Noun', '이/Josa', '넘도록/Verb', '음주운전/Noun', '차량/Noun', '을/Josa', '뒤/Noun', '쫓던/Verb', '경찰/Noun', '은/Josa', '결국/Adverb', '실탄/Noun', '을/Josa', '쏴/Verb', '차/Noun', '를/Josa', '세웠는데/Verb', '잡고/Noun', '보니이/Verb', '남성/Noun', '운전면허/Noun', '가/Josa', '없었습니다/Adjective', '경찰서/Noun', '경찰/Noun', '은/Josa', '도로/Noun', '교통법위반/Noun', '등/Noun', '의/Josa', '혐의/Noun', '로/Josa', '살/Noun', '씨/Suffix', '에/Josa', '대해/Noun', '구속영장/Noun', '을/Josa', '신청/Noun', '할/Verb', '계획/Noun', '입니다/Adjective']


# 정수화(Countvectorizer활용), TF-IDF로 중요 단어 추출

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

In [31]:
#형태소를 벡터 형태의 학습 데이터셋으로 변환
index_vectorizer = CountVectorizer(tokenizer = lambda x : get_pos(x))
X = index_vectorizer.fit_transform(df['ko_text'].tolist())

In [32]:
X.shape

(2858, 17641)

In [33]:
print(str(index_vectorizer.vocabulary_)[:100]+'..')

{'경찰/Noun': 947, '과/Josa': 1305, '추/Noun': 15200, '격전/Noun': 814, '까지/Josa': 2330, '벌인대/Verb': 6702,..


In [34]:
print(df['ko_text'][0])
print(X[0])

경찰과추격전까지벌인대남성이붙잡혔습니다시간이넘도록음주운전차량을뒤쫓던경찰은결국실탄을쏴시간이넘도록음주운전차량을뒤쫓던경찰은결국실탄을쏴차를세웠는데잡고보니이남성운전면허가없었습니다경찰서경찰은도로교통법위반등의혐의로살씨에대해구속영장을신청할계획입니다
  (0, 947)	4
  (0, 1305)	1
  (0, 15200)	1
  (0, 814)	1
  (0, 2330)	1
  (0, 6702)	1
  (0, 2676)	2
  (0, 11861)	3
  (0, 7525)	1
  (0, 9177)	2
  (0, 2903)	2
  (0, 11756)	2
  (0, 14652)	2
  (0, 11721)	5
  (0, 4439)	2
  (0, 14610)	2
  (0, 11668)	3
  (0, 841)	2
  (0, 9463)	2
  (0, 9589)	2
  (0, 14638)	1
  (0, 4959)	1
  (0, 8657)	1
  (0, 13059)	1
  (0, 6897)	1
  (0, 11285)	1
  (0, 2)	1
  (0, 10425)	1
  (0, 958)	1
  (0, 3896)	1
  (0, 1515)	1
  (0, 4615)	1
  (0, 11772)	1
  (0, 17145)	1
  (0, 4899)	1
  (0, 8016)	1
  (0, 9628)	1
  (0, 10449)	1
  (0, 3797)	1
  (0, 1585)	1
  (0, 9409)	1
  (0, 16546)	1
  (0, 1007)	1
  (0, 12709)	1


In [35]:
#중요 단어 추출을 위해 TF-IDF 활용
from sklearn.feature_extraction.text import TfidfTransformer

In [36]:
tfidf_vectorizer = TfidfTransformer()
X = tfidf_vectorizer.fit_transform(X)

In [37]:
print(X.shape)
print(X[0])

(2858, 17641)
  (0, 17145)	0.058432924416063285
  (0, 16546)	0.07304344251490438
  (0, 15200)	0.12818699469535608
  (0, 14652)	0.1706063226828352
  (0, 14638)	0.10035180375172324
  (0, 14610)	0.3553800152649298
  (0, 13059)	0.13744834722182506
  (0, 12709)	0.10981801750877301
  (0, 11861)	0.09944408616038823
  (0, 11772)	0.03347367695414807
  (0, 11756)	0.1898322514779223
  (0, 11721)	0.11815789676769288
  (0, 11668)	0.09600798206447946
  (0, 11285)	0.14103982383627867
  (0, 10449)	0.02756624188075282
  (0, 10425)	0.1776900076324649
  (0, 9628)	0.07122108376548993
  (0, 9589)	0.29577244818568227
  (0, 9463)	0.2861777729303359
  (0, 9409)	0.09785234750850758
  (0, 9177)	0.18153782387233097
  (0, 8657)	0.1776900076324649
  (0, 8016)	0.1201987923930827
  (0, 7525)	0.13916917621769598
  (0, 6897)	0.1776900076324649
  (0, 6702)	0.1776900076324649
  (0, 4959)	0.02785738229793186
  (0, 4899)	0.04240838682557293
  (0, 4615)	0.04287074626606472
  (0, 4439)	0.1651214743510662
  (0, 3896)	0.09193

# 모델 적용

In [38]:
from sklearn.model_selection import train_test_split

y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1)
print(X_train.shape)
print(X_test.shape)

(2572, 17641)
(286, 17641)


In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_pred_probability = lr.predict_proba(X_test)[:,1]

print("정확도:%.2f"% accuracy_score(y_test, y_pred))
print("정밀도:%.3f"% precision_score(y_test,y_pred, average = 'weighted'))
print("재현율:%.3f"% recall_score(y_test, y_pred, average = 'weighted'))
print("F1:%.3f"% f1_score(y_test,y_pred, average = 'weighted'))

정확도:0.92
정밀도:0.928
재현율:0.923
F1:0.921


param_lreg = { 'C' : [1.0,3,5,7,10], 
              'max_iter': [100,500]
             }
gscv_lreg = GridSearchCV(estimator = lreg, param_grid = param_lreg, scoring = 'accuracy', cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42), refit = True, n_jobs = 1, verbose =2)
gscv_lreg.fit(X_train, y_train)

print('lr 파라미터:', gscv_lreg.best_params_)
print('lr 예측 정확도:{:4f}'.format(gscv_lreg.best_score_))